### Stacked model

In [1]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import confinement_mode_classifier as cmc
from datetime import datetime
import time 
import torchvision
import torch
from torch.optim import lr_scheduler
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

import re

path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Seed set to 42


Device: cuda:0


In [2]:

data_dir_path = '/compass/Shared/Users/bogdanov/vyzkumny_ukol/data/LHmode-detection-shots'
file_names = os.listdir(data_dir_path)

batch_size = 16

#Chose what shots will be used in notebook. Removed shots has different dimensions
shot_numbers = [re.search(r'shot(\d+)', file_name).group(1) for file_name in file_names]
removed_shots = ['13182', '20009']
shot_numbers = [valid_shot for valid_shot in shot_numbers if valid_shot not in removed_shots]
shots_for_testing = ['18130', '16773', '16534', '19094', '18133']
shots_for_validation = ['16769', '19379', '18057', '18132']


shot_df, test_df, val_df, train_df = cmc.load_and_split_dataframes(path, shot_numbers, shots_for_testing, shots_for_validation, use_ELMS=False)

test_dataloader = cmc.get_two_imgs_dloader(test_df, path=path, batch_size=batch_size,
                                   shuffle=False, balance_data=True, second_img_opt='prev')

val_dataloader = cmc.get_two_imgs_dloader(val_df, path=path, batch_size=batch_size,
                                   shuffle=False, balance_data=True, second_img_opt='prev')

train_dataloader = cmc.get_two_imgs_dloader(train_df, path=path, batch_size=batch_size,
                                   shuffle=False, balance_data=True, second_img_opt='prev')

dataloaders = {'train':train_dataloader, 'val':val_dataloader}
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

In [3]:
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)

pretrained_model.load_state_dict(torch.load(f'{path}/runs/14-01-2024, 15-12-00 Train on new data, elms on_all_layers/model_fully_trained.pt'))

untrained_ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)

In [4]:
resnet_params = [resnet_name for resnet_name, resnet_param in pretrained_model.named_parameters()]

for name, param in untrained_ensembled_model.named_parameters():
    # Check if the current parameter is part of the MLP
    if 'classifier' in name or 'fc' in name or 'last_fully_connected' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Verify that only the MLP parameters have requires_grad set to True
for name, param in untrained_ensembled_model.named_parameters():
    if param.requires_grad:
        print(f"{name}: requires_grad = {param.requires_grad}")

modelA.fc.weight: requires_grad = True
modelA.fc.bias: requires_grad = True
modelB.fc.weight: requires_grad = True
modelB.fc.bias: requires_grad = True
classifier.0.weight: requires_grad = True
classifier.0.bias: requires_grad = True
classifier.2.weight: requires_grad = True
classifier.2.bias: requires_grad = True


In [5]:
timestamp =  datetime.fromtimestamp(time.time()).strftime("%d-%m-%Y, %H-%M-%S ") + input('add comment: ')
# create grid of images
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(f'runs/{timestamp}_classifier_training')

In [6]:
#
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(untrained_ensembled_model.parameters(), lr=0.01) #pouzit adam

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

num_epochs = 6
model_path = Path(f'{path}/runs/{timestamp}_classifier_training/model.pt')

ensembled_model = cmc.train_model(untrained_ensembled_model, criterion, optimizer, exp_lr_scheduler, 
                       dataloaders, writer, dataset_sizes, num_epochs=num_epochs, 
                       chkpt_path = model_path.with_name(f'{model_path.stem}_chkpt{model_path.suffix}'))

torch.save(ensembled_model.state_dict(), model_path)

Epoch 1/6
----------


  0%|          | 0/1003 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 314.00 MiB. GPU 0 has a total capacty of 10.76 GiB of which 248.44 MiB is free. Process 1900281 has 7.10 GiB memory in use. Process 2032028 has 2.25 GiB memory in use. Including non-PyTorch memory, this process has 1.17 GiB memory in use. Of the allocated memory 308.35 MiB is allocated by PyTorch, and 197.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from torch import cuda
if cuda.is_available():
    # Use this line if you have a single GPU
    cuda.empty_cache()
    
    # Use this line if you have multiple GPUs
    for i in range(cuda.device_count()):
        cuda.reset_max_memory_allocated(i)
        cuda.empty_cache()


untrained_ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)
untrained_ensembled_model.load_state_dict(torch.load(model_path))

writer = SummaryWriter(f'runs/{timestamp}_all_layers')

for name, param in untrained_ensembled_model.named_parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(untrained_ensembled_model.parameters(), lr=0.001) #pouzit adam

model_path = Path(f'{path}/runs/{timestamp}_all_layers/model.pt')

ensembled_model = cmc.train_model(untrained_ensembled_model, criterion, optimizer, exp_lr_scheduler, 
                       dataloaders, writer, dataset_sizes, num_epochs=num_epochs,
                         chkpt_path=model_path.with_name(f'{model_path.stem}_chkpt{model_path.suffix}'))


torch.save(ensembled_model.state_dict(), model_path)
